In [4]:
import os
import openai
import pandas as pd
from IPython.display import display, Markdown
import questionary

In [ ]:
openai.api_key = os.environ['SYS_OPENIA_API_KEY']

In [ ]:
class IPER_Row:
    '''
    Fila de la Matriz IPER.
    En términos simples, IPER es una descripción organizada de las actividades, 
    controles y peligros que permitan identificar los posibles riesgos. 
    Esta permitirá evaluar, monitorear, controlar y comunicar estos peligros 
    o sucesos no deseados, pudiendo también identificar los niveles de riesgo 
    y las consecuencias de estos.

    Los valores y definiciones de los distintos indicadores se describen a continuación:
        Índice de número de personas expuestas (self.INPE): Se determina en función de la cantidad de personas expuestas, en el lugar o entorno de trabajo.
        0: No aplica (NO existe exposición de personas, sólo de equipos/infraestructura)
        1: De 1 a 10 personas      
        2: De 11 a 20 personas
        3: Más de 20 personas

        Índice de Frecuencia y Duración de la exposición (self.IFDE): Considera dos variables, “Frecuencia” y “Duración” de la exposición según la siguiente tabla:
        Frecuencia de exposición del personal Duración de la exposición Índice 
        4: DIARIA o todos los días Menos del 50% del turno de trabajo
        5: DIARIA o todos los días Más del 50% del turno de trabajo
        3: SEMANALMENTE o todas las semanas (pero no se reitera cada día) Menos del 50% del turno de trabajo
        4: SEMANALMENTE o todas las semanas (pero no se reitera cada día) Más del 50% del turno de trabajo
        2: MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE Menos del 50% del turno de trabajo 
        3: MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE Más del 50% del turno de trabajo
        1: Se realiza con una frecuencia mayor a la SEMESTRAL Menos del 50% del turno de trabajo
        2: Se realiza con una frecuencia mayor a la SEMESTRAL Más del 50% del turno de trabajo
        2: Se está evaluando en Condiciones de Falla/no rutinaria o Anormales ó sólo se exponen equipos, no personas 

        Índice de Controles existentes (self.ICO)
        1: Existen controles implementados → Se tienen al menos tres condiciones implementadas (A, B, C ó D) 
        6: Existen controles parciales → Se tienen dos condiciones implementadas
        10: Los controles son bajos o insuficientes o no están implementados → 
        * Se cumple sólo una condición implementada o ninguna ó 
        * se identificaron "permanentes" desvíos comportamentales de incumplimiento a las normas referidas al peligro ó 
        * Es una evaluación "IPER anticipada para actividades" 

            Condiciones 
            Condicion A.  Existen ya implementados como parte del SGI procedimientos/documentos o controles operativos específself.ICOs para controlar el Peligro (p.e. PT, EPP, bloqueo/etiquetado, IT, guías, normas, inspecciones específicas).
            Condicion B.  Para el peligro específself.ICO existen implementados medios de infraestructura o ingeniería o protección colectiva o de emergencia para el Peligro (resguardos, barandas, cubiertas, líneas de vida, barreras de protección, aislamientos, medios de detección/alarma del peligro, dispositivos de enclavamiento/corte automátself.ICO, extintores, accesorios para mejorar la ergonomía, mantenimiento preventivo).
            Condicion C.  El personal ya tiene experiencia o fue capacitado/entrenado sobre el peligro específself.ICO al realizar la actividad.
            Condicion D. Se cuenta con señalización/alerta específica in-situ implementada para el peligro o situación peligrosa.

        Criterios de Severidad de Daño:
        1: "Daño Menor"
            - Lesiones que sólo requieren primeros auxilios o atención médica de seguimiento.
            - Lesiones/enfermedades que ocasionan ausencia laboral de menos de un día o transferencia de actividad por ese período.
            - Daños a equipos con costos estimados menores a 1,000 USD.

        2: "Daño Mediano"
            - Lesiones/enfermedades que ocasionan ausencia laboral temporal de 1 día a 1 mes o transferencia de actividad por ese período.
            - Daños a equipos con costos entre 1,001 y 10,000 USD.

        3: "Daño Mayor"
            - Lesiones que ocasionan ausencia laboral temporal de 1 mes a 6 meses o transferencia de actividad por ese período.
            - Lesiones/enfermedades que ocasionan "incapacidades permanentes parciales".
            - Daños a equipos con costos entre 10,001 y 100,000 USD.

        4: "Daño Extremo"
            - Lesiones/enfermedades que ocasionan ausencia laboral temporal de 6 meses a 1 año o transferencia de actividad por ese período.
            - Lesiones o enfermedades que generan "incapacidad total" al trabajador.
            - Muerte.
            - Daños a equipos con costos estimados superiores a 100,000 USD.
    '''
    def __init__(self, sector, subsector, person_count, usage_time, implemented_controls,  problem_description):
        
        self.sector = sector # string
        self.subsector = subsector # string
        self.problem_description = problem_description # string
        self.implemented_controls = implemented_controls # list
        self.person_count = person_count #int
        self.usage_time = usage_time #int horas-semana

        # Indices
        self.INPE = self._get_inpe()
        self.IFDE = self._get_ifde() 
        self.ICO = self._get_ico()

        self.probabilidad = self._evaluar_probabilidad()
        self.nivel_de_riesgo = self._calcular_nivel_riesgo()
        self.aceptabilidad = self._evaluar_aceptabilidad()

        self.severidad_daño = self._get_severidad()
        self.string_severidad_daño = _get_severidad_daño()

        self.condicion_evaluacion = self._elegir_condicion()
        self.peligro = self._seleccionar_peligro()
        self.origen_peligro = self._generar_origen_peligrp()
        self.controles_pre_existentes = self._generar_controles_pre_existentes() 

        self.opciones_peligro = {
            1: "A1. Caída de personas al mismo nivel",
            2: "A2. Caídas menores a distinto nivel (entre 0,3 y 1,8 m)",
            3: "A3. Caídas mayores a distinto nivel (mayor a 1,8 m)",
            4: "A4. Contactos eléctricos (Choque eléctrico)",
            5: "A5. Contactos con partes o elementos calientes/fríos",
            6: "A6. Proyección de partículas, fragmentos",
            7: "A7. Proyección de gases, polvo o líquidos a presión ó calientes",
            8: "A8. Atrapamientos mecánicos",
            9: "A9. Cortes, golpes, penetraciones por herramientas",
            10: "A10. Cortes, golpes, penetraciones, excoriaciones de otra clase (no por herramientas)",
            11: "A11. Caída de objetos menores (menos de 5 kg) o herramientas",
            12: "A12. Aplastamiento/Ahogamiento (entre objetos o por caída/deslizamiento de objetos mayores a 5 Kg)",
            13: "A13. Golpes por objetos/equipos móviles o atropellamiento por vehículos",
            14: "A14. Golpes por objetos inmóviles o partes salientes",
            15: "A15. Incendios",
            16: "A16. Explosiones / deflagraciones",
            17: "A17. Choques de vehículos en movimiento",
            18: "A18. Vuelcos vehiculares o de equipo",
            19: "A19. Exposición a ruido",
            20: "A20. Exposición a vibraciones",
            21: "A21. Exposición a inadecuada iluminación",
            22: "A22. Exposición a temperaturas extremas (extremadamente mayor a la normal o menor a 0°C)",
            23: "A23. Exposición a humedad extrema",
            24: "A24. Exposición a radiaciones ionizantes",
            25: "A25. Exposición a radiaciones no ionizantes",
            26: "B1. Contacto o ingestión de sólidos/líquidos peligrosos",
            27: "B2. Exposición a polvos o fibras",
            28: "B3. Exposición a gases/vapores tóxicos o asfixiantes",
            29: "B4. Derrames o fugas mayores de sustancias peligrosas",
            30: "B5. Exposición a insectos/animales peligrosos",
            31: "B6. Exposición a bacterias, virus u hongos",
            32: "C1. Ejecución de posturas inadecuadas",
            33: "C2. Ejecución de movimientos repetitivos",
            34: "C3. Ejecución de sobre esfuerzo físicos",
            35: "C4. Exposición a sobre esfuerzo visual",
            36: "C5. Exposición a sobre esfuerzo mental",
            37: "D1. Sismos",
            38: "D2. Inundaciones (por lluvias o granizadas intensas o desborde de ríos)",
            39: "D3. Tormentas eléctricas o de vientos huracanados",
            40: "D4. Deslizamientos de tierra",
            41: "D5. Incendios de plantas aledañas o forestales",
            42: "D6. Convulsión social"
        }

    def _evaluar_probabilidad(self):
        '''
        Evalúa la probabilidad de riesgo en función de los indicadores.

        Args:
            self.INPE (int): Índice de número de personas expuestas. (0-3)
            self.IFDE (int): Índice de Frecuencia y Duración de la exposición. (1-5)
            self.ICO (int): Índice de Controles existentes. (1, 6, 10)

        Returns:
            int: Valor de la probabilidad de riesgo (1-5) o None si no se cumple ninguna condición.
        '''
        suma = self.INPE + self.IFDE + self.ICO
        if suma <= 6:
            return 1
        elif suma <= 10:
            return 2
        elif suma <= 14:
            return 3
        elif suma <= 18:
            return 4
        elif suma > 18:
            return 5
        else:
            return None

    def _calcular_nivel_riesgo(self):
        '''
        Calcula el nivel de riesgo en función de la probabilidad y la severidad del daño.

        Args:
            self.probabilidad (int): Valor de la probabilidad de riesgo (1-5).
            self.severidad_daño (int): Valor de la severidad del daño (1-5).

        Returns:
            str: Nivel de riesgo ("Trivial", "Bajo", "Moderado", "Alto" o "Intolerable").
        '''
        resultado = self.probabilidad * self.severidad_daño
        nivel = ""
        if resultado <= 2:
            nivel = "Trivial"
        elif resultado <= 6:
            nivel = "Bajo"
        elif resultado <= 9:
            nivel = "Moderado"
        elif resultado <= 15:
            nivel = "Alto"
        elif resultado >= 16:
            nivel = "Intolerable"

        return nivel

    def _evaluar_aceptabilidad(self):
        '''
        Evalúa la aceptabilidad del riesgo en función de la probabilidad y la severidad del daño.

        Args:
            self.probabilidad (int): Valor de la probabilidad de riesgo (1-5).
            self.severidad_daño (int): Valor de la severidad del daño (1-5).

        Returns:
            str: Aceptabilidad del riesgo ("Aceptable" o "No Aceptable").
        '''
        resultado = self.probabilidad * self.severidad_daño
        nivel = ""
        if resultado > 7:
            nivel = "No Aceptable"
        else:
            nivel = "Aceptable"

        return nivel

    def _elegir_condicion(self):

        # Define the prompt using self.problem_description
        prompt = f"""Selecciona la condición más adecuada basada en la siguiente descripción:

        Descripción del problema: 
        {self.problem_description}

        Opciones de condición:
        0. Normal/Rutinaria
        1. Anormal/Emergencia/No rutinaria
        """

        # Generate a response using the ChatGPT API
        response = openai.Completion.create(
            engine='davinci',
            prompt=prompt,
            max_tokens=1,
            n=1,
            stop=None,
            temperature=0,
        )

        # Extract the selected option (con_eval) from the generated response
        con_eval = int(response.choices[0].text.strip())

        if con_eval == 0:
            return "Normal/Rutinaria"
        elif con_eval == 1:
            return "Anormal/Emergencia/No rutinaria"
        else:
            return None

    def _seleccionar_peligro(self):
        # Define the prompt using self.problem_description and opciones_peligro
        prompt = f"""Selecciona el peligro más adecuado basado en la siguiente descripción:

        Descripción del problema: {self.problem_description}

        Opciones de peligro:
        {self.opciones_peligro}
        """

        # Generate a response using the ChatGPT API
        response = openai.Completion.create(
            engine='davinci',
            prompt=prompt,
            max_tokens=1,
            n=1,
            stop=None,
            temperature=0,
        )

        # Extract the selected option (key) from the generated response
        selected_option = int(response.choices[0].text.strip())

        return selected_option

    def _generar_origen_peligro(self):
        pass

    def _generar_controles_pre_existentes(self):
        pass
    
    def _get_severidad(self):
        danio_menor = [1, 2, 5, 9, 10, 14, 19, 20, 21, 22, 23, 25, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42]
        danio_mediano = [3, 4, 7, 8, 13, 17, 18, 24, 27, 37]
        danio_mayor = [6, 11, 16, 26, 38]
        danio_extremo = [15, 25]
        if self.peligro in danio_menor:
            return 1 
        elif self.peligro in danio_mediano:
            return 2 
        elif self.peligro in danio_mayor:
            return 3 
        elif self.peligro in danio_extremo:
            return 4
        else:
            return 0
    
    def _get_inpe(self):
        opciones_cantidad = [
            'No aplica (NO existe exposición de personas, sólo de equipos/infraestructura',
            'De 1 a 10 personas',
            'De 11 a 20 personas',
            'Más de 20 personas'
            ]

        cantidad = select('Selecciona una opción', choices=opciones_cantidad).ask()

        if cantidad == 'De 1 a 10 personas':
            return 1
        elif cantidad == 'De 11 a 20 personas':
            return 3
        elif cantidad == 'Más de 20 personas':
            return 5
        else:
            return None

        
    def _get_ifde(self):
        opciones_frecuencia = [
        'DIARIA o todos los días',
        'SEMANALMENTE o todas las semanas (pero no se reitera cada día)',
        'MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE',
        'Se realiza con una frecuencia mayor a la SEMESTRAL',
        'Se está evaluando en Condiciones de Falla/no rutinaria o Anormales ó sólo se exponen equipos, no personas'
        ]

        opciones_duracion = [
            'Menos del 50% del turno de trabajo',
            'Más del 50% del turno de trabajo'
            ]

        frecuencia = select('Selecciona una opción', choices=opciones_frecuencia).ask()

        if frecuencia == 'DIARIA o todos los días':
            duracion = select('Selecciona una opción', choices=opciones_duracion).ask()
            if duracion == 'Menos del 50% del turno de trabajo':
                return 4
            else:
                return 5
        elif frecuencia == 'SEMANALMENTE o todas las semanas (pero no se reitera cada día)':
            duracion = select('Selecciona una opción', choices=opciones_duracion).ask()
            if duracion == 'Menos del 50% del turno de trabajo':
                return 3
            else:
                return 4
        elif frecuencia == 'MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE':
            duracion = select('Selecciona una opción', choices=opciones_duracion).ask()
            if duracion == 'Menos del 50% del turno de trabajo':
                return 2
            else:
                return 3
        elif frecuencia == 'Se realiza con una frecuencia mayor a la SEMESTRAL':
            duracion = select('Selecciona una opción', choices=opciones_duracion).ask()
            if duracion == 'Menos del 50% del turno de trabajo':
                return 1
            else:
                return 2
        elif frecuencia == 'Se está evaluando en Condiciones de Falla/no rutinaria o Anormales ó sólo se exponen equipos, no personas':
            return 2
        else:
            return None

        def _get_ico(self):
            x = len(self.implemented_controls)
            if x => 3:
                return 1 
            elif x == 2:
                return 6
            elif x == 1:
                return 10
        

# Create an instance of Iper_Row
iper_row = IPER_Row(INPE=2, IFDE=4, ICO=1, severidad_daño=3)
iper_row.probabilidad

In [3]:
from questionary import select


def obtener_IFDE():
    opciones_frecuencia = [
        'DIARIA o todos los días',
        'SEMANALMENTE o todas las semanas (pero no se reitera cada día)',
        'MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE',
        'Se realiza con una frecuencia mayor a la SEMESTRAL',
        'Se está evaluando en Condiciones de Falla/no rutinaria o Anormales ó sólo se exponen equipos, no personas'
    ]

    opciones_duracion = [
        'Menos del 50% del turno de trabajo',
        'Más del 50% del turno de trabajo',
    ]

    frecuencia = input('Selecciona la frecuencia de exposición del personal: ')
    duracion = input('Selecciona la duración de la exposición: ')

    if frecuencia == 'DIARIA o todos los días':
        if duracion == 'Menos del 50% del turno de trabajo':
            return 4
        else:
            return 5
    elif frecuencia == 'SEMANALMENTE o todas las semanas (pero no se reitera cada día)':
        if duracion == 'Menos del 50% del turno de trabajo':
            return 3
        else:
            return 4
    elif frecuencia == 'MENSUALMENTE (pero no se reitera cada semana) a SEMESTRALMENTE':
        if duracion == 'Menos del 50% del turno de trabajo':
            return 2
        else:
            return 3
    elif frecuencia == 'Se realiza con una frecuencia mayor a la SEMESTRAL':
        if duracion == 'Menos del 50% del turno de trabajo':
            return 1
        else:
            return 2
    elif frecuencia == 'Se está evaluando en Condiciones de Falla/no rutinaria o Anormales ó sólo se exponen equipos, no personas':
        return 0
    else:
        return None


resultado = obtener_IFDE()
if resultado is not None:
    print(f'Índice de Frecuencia y Duración de la exposición (self.IFDE): {resultado}')
else:
    print('Opción inválida')

Opción inválida


In [19]:
# Configure pandas display options
pd.set_option('display.max_rows', None)     # Print all rows
pd.set_option('display.max_columns', None)  # Print all columns

# Read the Excel file
file_path = 'IPER_example.xlsx'
df = pd.read_excel(file_path)
# Drop rows with NaN values
df = df.dropna()

df.head()

,#,SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO,ACTIVIDAD / TAREA / LUGAR / EQUIPO / EVENTO,CONDICIÓN DE EVALUACIÓN,"PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)",CONSECUENCIAS_x000D_\nmas probables_x000D_\n_x000D_\n(Lesiones o daños mas probables),"DESVIO O CAUSA QUE ORIGINA EL PELIGRO_x000D_\n¿Por qué se genera el peligro?_x000D_\n_x000D_\nCausas/Devío: (Condiciones inseguras / Factores inseguros del Trabajo / Deficiencias de seguridad/ Actos Inseguros que generan el peligro)_x000D_\n_x000D_\nElementos: (Energías, equipos, maquinarias, sustancias, etc.)",DETERMINACIÓN DE LOS CONTROLES O PROTECCIÓNES EXISTENTES_x000D_\n_x000D_\n(Determine aquellos controles preventivos o de protección actualmente existentes),SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO,SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO.1,INPE,IFDE,ICO,∑,Prob.,Prob..1,NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE,ES REQUISITO LEGAL? CUAL,MEDIDAS ADICIONALES DE CONTROL O PROTECCIÓN PROPUESTAS_x000D_\n_x000D_\n(Debe atacar las CAUSAS identificadas)_x000D_\n_x000D_\n1. Eliminacion: _x000D_\n2. Sustitucion: _x000D_\n3.Controles Ingenieria/Protección colectiva:_x000D_\n4. Señalización/Controles administrativos: _x000D_\n5. EPP:,SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución),SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución).1,INPE.1,IFDE.1,ICO.1,∑.1,Prob..2,Prob..3,NUEVO NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE.1
1,2,1er Piso,OF SUB GERENCIA TECNICA,Anormal/Emergencia/No rutinaria,A15. Incendios,"Daños a instalaciones y/o equipos, quemaduras,...","Incendio por descargas eléctricas, cortos circ...",1. Capacitacion de prevención de incendios con...,Daño Extremo,4,1,2,10,13,Media,3,Alto,No Aceptable,"SI, Ley 16998 Art. 123",1. Realizar Mantenimiento preventivo de las in...,Daño Extremo,4.0,1.0,2.0,1.0,4.0,Muy Baja,1.0,Bajo,Aceptable
4,5,1er Piso,PASILLO,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Superficie de área de trabajo en mal estado (A...,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,"SI, Ley Nº 16998 Art 58",4. Mejorar las condiciones de las alfombras\n4...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
5,6,1er Piso,PASILLO,Normal/Rutinaria,A2. Caídas menores a distinto nivel (entr...,"Traumatismos, heridas, fracturas múltiples","Uso contínuo de gradas fijas, deficiente ilumi...",1. Monitoreo de iluminancia,Daño Mayor,3,1,5,10,16,Alta,4,Alto,No Aceptable,SI,"1. Realizar inspecciones de orden, limpieza y ...",Daño Mayor,3.0,1.0,5.0,1.0,7.0,Baja,2.0,Bajo,Aceptable
6,7,1er Piso,PASILLO,Anormal/Emergencia/No rutinaria,A4. Contactos eléctricos (Choque eléctrico),"Quemaduras internas, externas, arritmia y paro...",El tablero eléctrico no tiene seguro (acceso a...,NINGUNO,Daño Mayor,3,1,2,10,13,Media,3,Moderado,No Aceptable,SI\nLey Nº 16998 Art 123-130,3. Colocar candado o chapa\n3. Realizar manten...,Daño Mayor,3.0,1.0,2.0,6.0,9.0,Baja,2.0,Bajo,Aceptable
7,8,1er Piso,PASILLO,Anormal/Emergencia/No rutinaria,A15. Incendios,"Daños a instalaciones y/o equipos, quemaduras,...","Incendio por descargas, corto circuitos, Insta...",1. Capacitacion de prevención de incendios con...,Daño Extremo,4,1,2,10,13,Media,3,Alto,No Aceptable,SI,1. Instalación de Termico\n3. Instalación de e...,Daño Extremo,4.0,1.0,2.0,1.0,4.0,Muy Baja,1.0,Bajo,Aceptable


In [23]:
df.sort_values(by=['PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)'])

,#,SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO,ACTIVIDAD / TAREA / LUGAR / EQUIPO / EVENTO,CONDICIÓN DE EVALUACIÓN,"PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)",CONSECUENCIAS_x000D_\nmas probables_x000D_\n_x000D_\n(Lesiones o daños mas probables),"DESVIO O CAUSA QUE ORIGINA EL PELIGRO_x000D_\n¿Por qué se genera el peligro?_x000D_\n_x000D_\nCausas/Devío: (Condiciones inseguras / Factores inseguros del Trabajo / Deficiencias de seguridad/ Actos Inseguros que generan el peligro)_x000D_\n_x000D_\nElementos: (Energías, equipos, maquinarias, sustancias, etc.)",DETERMINACIÓN DE LOS CONTROLES O PROTECCIÓNES EXISTENTES_x000D_\n_x000D_\n(Determine aquellos controles preventivos o de protección actualmente existentes),SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO,SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO.1,INPE,IFDE,ICO,∑,Prob.,Prob..1,NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE,ES REQUISITO LEGAL? CUAL,MEDIDAS ADICIONALES DE CONTROL O PROTECCIÓN PROPUESTAS_x000D_\n_x000D_\n(Debe atacar las CAUSAS identificadas)_x000D_\n_x000D_\n1. Eliminacion: _x000D_\n2. Sustitucion: _x000D_\n3.Controles Ingenieria/Protección colectiva:_x000D_\n4. Señalización/Controles administrativos: _x000D_\n5. EPP:,SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución),SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución).1,INPE.1,IFDE.1,ICO.1,∑.1,Prob..2,Prob..3,NUEVO NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE.1
33,34,Patio,Parqueo,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...","Existencia de desniveles, exceso de maleza,",NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,4. Programa de mantenimiento (limpieza de male...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
97,98,Almacen Sopocachi,Almacenes,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Poco espacio para circular; Material en el Piso,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,1. Mejorar la disposicion de los insumos y mat...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
40,41,Planta Baja,Oficina Jefatura Administracion Y Finanzas,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Superficie de área de trabajo en mal estado (A...,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,1. Mantenimiento de alfombra\n1. Realizar insp...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
23,24,Patio,Patio trasero,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...","Existencia de desniveles, falta de limpieza, m...",NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,4. Programa de mantenimiento del jardín (limpi...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
45,46,Planta Baja,Area de Producción y Mensajeria,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Superficie de área de trabajo en mal estado (A...,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,1. Mantenimiento de alfombra\n1. Realizar insp...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
128,129,Logistica,Almacen El Alto,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Poco espacio para circular (1er piso delantero...,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,1. Mejorar la disposicion de los insumos y mat...,Daño Mediano,2.0,1.0,5.0,6.0,12.0,Media,3.0,Bajo,Aceptable
50,51,Planta Baja,Area de Recepcion y Comercializacion,Normal/Rutinaria,A1. Caída de personas al mismo nivel,"Traumatismos, heridas, Golpes, contusiones, e...",Superficie de área de trabajo en mal estado (A...,NINGUNO,Daño Mediano,2,1,5,10,16,Alta,4,Moderado,No Aceptable,SI,1. Mantenimiento de alfombra\n4. Señalizacio

In [14]:
# Read the Excel file
file_path = 'PG_example.xlsx'
pg = pd.read_excel(file_path)
# Drop rows with NaN values
pg = df.dropna()

pg.head()

,#,# PLANILLA IDENTIFICACIÓN Y EVALUACIÓN,OBJETIVO,PELIGRO RELACIONADO,META,ACTIVIDAD,RESPONSABLE EJECUCIÓN,RECURSOS\n(Bs),PLAZOS (Dias),RESPONSABLE SEGUIMIENTO,AVANCE DE SEGUIMIENTO %
0,1,"5,15, 41, 46, 51, 70","Prevenir traumatismos, heridas, Golpes, contus...",A1. Caída de personas al mismo nivel\n\n,100 % de Acividades Ejecutadas,Mejorar las condiciones de las superficies de ...,Jenny Chuquimia,500,41390,Mirko Taquichiri,100
11,12,40,"Prevenir traumatismos, heridas, Golpes, contus...","A2. Caídas menores a distinto nivel (entre 0,3...",100 % de Acividades Ejecutadas,Realizar Matenimiento de la baranda de gradas ...,Jenny Chuquimia,30,41425,Ingrid Terrazas,100
19,20,7,Prevenir lesiones ocasionadas por contactos el...,A4. Contactos eléctricos (Choque eléctrico)\nA...,100 % de Acividades Ejecutadas,Colocar candado de proteccion al tablero eléct...,Jenny Chuquimia,50,41432,Mirko Taquichiri,13
29,30,19,Prevenir lesiones ocasionadas por atrapamiento...,A8. Atrapamientos mecánicos,100 % de Acividades Ejecutadas,Cambiar la posición del ventilador a una mas a...,Cecilia Guachalla,150,41445,Mirko Taquichiri,18
34,35,"9, 17, 42, 52, 66, 71","Prevenir golpes, contusiones y fracturas",A11. Caída de objetos menores (menos de 5 kg) ...,100 % de Acividades Ejecutadas,Instalar protección contra caida necesaria a l...,Jenny Chuquimia,3400,41425,Mirko Taquichiri,100


In [18]:
iper = pd.DataFrame(columns=['#', 'SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO',
       'ACTIVIDAD / TAREA / LUGAR / EQUIPO / EVENTO',
       'CONDICIÓN DE EVALUACIÓN',
       'PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)',
       'CONSECUENCIAS_x000D_\nmas probables_x000D_\n_x000D_\n(Lesiones o daños mas probables)',
       'DESVIO O CAUSA QUE ORIGINA EL PELIGRO_x000D_\n¿Por qué se genera el peligro?_x000D_\n_x000D_\nCausas/Devío: (Condiciones inseguras / Factores inseguros del Trabajo / Deficiencias de seguridad/ Actos Inseguros que generan el peligro)_x000D_\n_x000D_\nElementos: (Energías, equipos, maquinarias, sustancias, etc.)',
       'DETERMINACIÓN DE LOS CONTROLES O PROTECCIÓNES EXISTENTES_x000D_\n_x000D_\n(Determine aquellos controles preventivos o de protección actualmente existentes)',
       'SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO',
       'SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO.1', 'INPE', 'IFDE', 'ICO',
       '∑', 'Prob.', 'Prob..1', 'NIVEL DEL RIESGO',
       'ACEPTABLE / NO ACEPTABLE', 'ES REQUISITO LEGAL? CUAL',
       'MEDIDAS ADICIONALES DE CONTROL O PROTECCIÓN  PROPUESTAS_x000D_\n_x000D_\n(Debe atacar las CAUSAS identificadas)_x000D_\n_x000D_\n1. Eliminacion: _x000D_\n2. Sustitucion: _x000D_\n3.Controles Ingenieria/Protección colectiva:_x000D_\n4. Señalización/Controles administrativos: _x000D_\n5. EPP:',
       'SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución)',
       'SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución).1',
       'INPE.1', 'IFDE.1', 'ICO.1', '∑.1', 'Prob..2', 'Prob..3',
       'NUEVO NIVEL DEL RIESGO', 'ACEPTABLE / NO ACEPTABLE.1'])

iper.head()

,#,SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO,ACTIVIDAD / TAREA / LUGAR / EQUIPO / EVENTO,CONDICIÓN DE EVALUACIÓN,"PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)",CONSECUENCIAS_x000D_\nmas probables_x000D_\n_x000D_\n(Lesiones o daños mas probables),"DESVIO O CAUSA QUE ORIGINA EL PELIGRO_x000D_\n¿Por qué se genera el peligro?_x000D_\n_x000D_\nCausas/Devío: (Condiciones inseguras / Factores inseguros del Trabajo / Deficiencias de seguridad/ Actos Inseguros que generan el peligro)_x000D_\n_x000D_\nElementos: (Energías, equipos, maquinarias, sustancias, etc.)",DETERMINACIÓN DE LOS CONTROLES O PROTECCIÓNES EXISTENTES_x000D_\n_x000D_\n(Determine aquellos controles preventivos o de protección actualmente existentes),SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO,SEVERIDAD_x000D_\n_x000D_\nDEL DAÑO.1,INPE,IFDE,ICO,∑,Prob.,Prob..1,NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE,ES REQUISITO LEGAL? CUAL,MEDIDAS ADICIONALES DE CONTROL O PROTECCIÓN PROPUESTAS_x000D_\n_x000D_\n(Debe atacar las CAUSAS identificadas)_x000D_\n_x000D_\n1. Eliminacion: _x000D_\n2. Sustitucion: _x000D_\n3.Controles Ingenieria/Protección colectiva:_x000D_\n4. Señalización/Controles administrativos: _x000D_\n5. EPP:,SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución),SEVERIDAD RESIDUAL _x000D_\nDEL DAÑO_x000D_\n_x000D_\n(analizar si hay disminución).1,INPE.1,IFDE.1,ICO.1,∑.1,Prob..2,Prob..3,NUEVO NIVEL DEL RIESGO,ACEPTABLE / NO ACEPTABLE.1


In [17]:
pg = pd.DataFrame(columns=['#', '# PLANILLA IDENTIFICACIÓN  Y EVALUACIÓN', 'OBJETIVO',
       'PELIGRO RELACIONADO', 'META', 'ACTIVIDAD',
       'RESPONSABLE EJECUCIÓN', 'RECURSOS\n(Bs)', 'PLAZOS (Dias)',
       'RESPONSABLE SEGUIMIENTO', 'AVANCE DE  SEGUIMIENTO %'])

pg.head()

,#,# PLANILLA IDENTIFICACIÓN Y EVALUACIÓN,OBJETIVO,PELIGRO RELACIONADO,META,ACTIVIDAD,RESPONSABLE EJECUCIÓN,RECURSOS\n(Bs),PLAZOS (Dias),RESPONSABLE SEGUIMIENTO,AVANCE DE SEGUIMIENTO %


In [27]:
class gestion_program(pd.DataFrame):
    def __init__(self, IPER):
        peligros = IPER['PELIGRO_x000D_\n_x000D_\n(Evento Peligroso, categorias de Lista Maestra OST-SGI.SI.001)'].unique()
        indices = IPER['#'].unique()
        sectores = IPER['SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO'].unique()
        
        # Crear DataFrame con las columnas especificadas
        super().__init__(columns=['#', '# PLANILLA IDENTIFICACIÓN  Y EVALUACIÓN', 'OBJETIVO',
                                 'PELIGRO RELACIONADO', 'META', 'ACTIVIDAD',
                                 'RESPONSABLE EJECUCIÓN', 'RECURSOS\n(Bs)', 'PLAZOS (Dias)',
                                 'RESPONSABLE SEGUIMIENTO', 'AVANCE DE  SEGUIMIENTO %'])
        
        # Generar filas por combinación de sector y peligro
        for peligro in peligros:
            for indice in indices:
                sector = IPER.loc[IPER['#'] == indice, 'SECTOR / AREA / UNIDAD / PROCESO O SUB-PROCESO'].values[0]
                objetivo = "Generar objetivo para todas las filas que compartan sector y peligro"
                actividad = "Generar con la API de OpenAI ChatGPT una actividad enfocada en resolver los peligros en ese sector, especificando los subsectores de cada índice involucrado."
                
                self.loc[len(self)] = [str(indice), ','.join(map(str, indices)), objetivo, peligro, "100 % de Actividades Ejecutadas",
                                       actividad, '', '', '', '', '0.00']


In [28]:
pg_1=gestion_program(df)

In [29]:
pg_1.head()

,#,# PLANILLA IDENTIFICACIÓN Y EVALUACIÓN,OBJETIVO,PELIGRO RELACIONADO,META,ACTIVIDAD,RESPONSABLE EJECUCIÓN,RECURSOS\n(Bs),PLAZOS (Dias),RESPONSABLE SEGUIMIENTO,AVANCE DE SEGUIMIENTO %
0,2,"2,5,6,7,8,9,13,15,16,17,18,19,21,22,24,25,26,2...",Generar objetivo para todas las filas que comp...,A15. Incendios,100 % de Actividades Ejecutadas,Generar con la API de OpenAI ChatGPT una activ...,,,,,0.00
1,5,"2,5,6,7,8,9,13,15,16,17,18,19,21,22,24,25,26,2...",Generar objetivo para todas las filas que comp...,A15. Incendios,100 % de Actividades Ejecutadas,Generar con la API de OpenAI ChatGPT una activ...,,,,,0.00
2,6,"2,5,6,7,8,9,13,15,16,17,18,19,21,22,24,25,26,2...",Generar objetivo para todas las filas que comp...,A15. Incendios,100 % de Actividades Ejecutadas,Generar con la API de OpenAI ChatGPT una activ...,,,,,0.00
3,7,"2,5,6,7,8,9,13,15,16,17,18,19,21,22,24,25,26,2...",Generar objetivo para todas las filas que comp...,A15. Incendios,100 % de Actividades Ejecutadas,Generar con la API de OpenAI ChatGPT una activ...,,,,,0.00
4,8,"2,5,6,7,8,9,13,15,16,17,18,19,21,22,24,25,26,2...",Generar objetivo para todas las filas que comp...,A15. Incendios,100 % de Actividades Ejecutadas,Generar con la API de OpenAI ChatGPT una activ...,,,,,0.00


In [30]:
pg_1.info()

<class '__main__.gestion_program'>
Int64Index: 2185 entries, 0 to 2184
Data columns (total 11 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   #                                        2185 non-null   object
 1   # PLANILLA IDENTIFICACIÓN  Y EVALUACIÓN  2185 non-null   object
 2   OBJETIVO                                 2185 non-null   object
 3   PELIGRO RELACIONADO                      2185 non-null   object
 4   META                                     2185 non-null   object
 5   ACTIVIDAD                                2185 non-null   object
 6   RESPONSABLE EJECUCIÓN                    2185 non-null   object
 7   RECURSOS
(Bs)                            2185 non-null   object
 8   PLAZOS (Dias)                            2185 non-null   object
 9   RESPONSABLE SEGUIMIENTO                  2185 non-null   object
 10  AVANCE DE  SEGUIMIENTO %                 2185 non-null   object